In [1]:
import urllib
from urllib.parse import urlparse, parse_qs, urlunparse, urlsplit
from bs4 import BeautifulSoup
from datetime import datetime

In [24]:
#a random thread starter
url = 'https://www.notaryrotary.com/forums/forums.asp?id=&forumid=1AAA00000003&messageid=2081512&code=&smsg=&requestid=&action=view&format=threaded'
#url = 'https://www.notaryrotary.com/forums/forums.asp?id=&forumid=1AAA00000003&messageid=2081526&code=&smsg=&requestid=&action=view&format=threaded'

In [163]:
class Post:
    
    regex = re.compile("/images/sp.\.gif")
    
    def __init__(self, url):
        self.url = url
        self.parsed_url = urlparse(url)
        self.forum_id = None
        self.message_id = None
        self.author = None
        self.content = None
        self.title = None
        self.time = None
        self.children = []
        
    #process the URL and turn it into soup
    def make_soup(self):
        my_url = self.url
        req = urllib.request.Request(my_url)
        response = urllib.request.urlopen(req)
        thePage = response.read()
        return BeautifulSoup(thePage, 'lxml')

    '''grab post data'''
    def fill_post(self):
        #pull in data
        my_soup = self.make_soup()
        
        
        #get forum_id
        query = parse_qs(self.parsed_url.query)
        self.forum_id = query['forumid'][0]
        self.message_id = int(query['messageid'][0])

        #get title
        self.title = my_soup.find('title').get_text()

        #get content
        self.content = my_soup.find(class_ = 'ForumText').get_text()

        #get author.  There are multiple <b>, but the first is the author
        self.author = my_soup.find('b').get_text()

        #get timestamp
        my_time = my_soup.find('em').get_text()
        my_time = my_time.strip(' ')
        my_time_real = datetime.strptime(my_time, '%m/%d/%y %I:%M%p')
        self.time = datetime.strftime(my_time_real, '%Y-%m-%d %H:%M:00')
        
        #bring in children
        self.rest_of_thread(my_soup)

    '''all other posts in this thread'''        
    def rest_of_thread(self, my_soup):
        #TDAlt9 is unique for the messages table, specifically a cell in it
        t = my_soup.find(class_ = 'TDAlt9')
        thread_table = t.parent.parent
        for r in thread_table.find_all('tr'):
            i = r.find('img')
            try:   #first row has no image
                if re.match('/images/sp.\.gif', i['src']):   #children are sp1, grandchildren sp2, etc
                    ra = r.find('a', class_='A2')     #profiles have links, but not A2 class
                    rah = ra['href']
                    parsed_rah = urlparse(rah)
                    rah_query = parse_qs(parsed_rah.query)
                    self.children.append(tuple((rah_query['forumid'][0], rah_query['messageid'][0])))
            except:
                continue

    '''print for validation'''
    def display(self):
        print(self.author + ' @ ' + self.time)
        print('==='+self.title+'===')
        print(self.content)
        print('')


In [165]:
def display_thread(my_post):
    my_post.display()
    try:
        for c in my_post.children:
            my_query = 'forumid=%s&messageid=%s&action=view&format=threaded' % c
            my_url = urlunparse((my_post.parsed_url.scheme, my_post.parsed_url.netloc, my_post.parsed_url.path, '', my_query, ''))
            my_child = Post(my_url)
            my_child.fill_post()
            my_child.display()
    except:
        print('none')

In [167]:
a_post = Post(url)
a_post.fill_post()

display_thread(a_post)

Susan/CA @ 2017-05-25 15:09:00
===PDF files scanned are too large to send in emails. How can..===
I recently invested in a scanner/all in one because I have been getting a lot of full package scan back requests the past several months. I was faxing before. However, when I scan a set of documents (I have also tried breaking up the package into 2 or 3 sections then sending each pdf seperately) then try to email them (on AOL) each time it says the file is too large. How do you all get a large package sent? Is it an AOL issue or should I scan in a different format (JPEG etc.) or better yet how do I create a zip file or some other file so I can sent large scans?Thank you sooooo much!!! 

 Negrete @ 2017-05-25 15:16:00
===Re: PDF files scanned are too large to send in emails. How can..===
AOL is a PIA for trying to send any scanned docs.

 sigtogo/OR @ 2017-05-25 15:22:00
===scanner settings make all the difference for me===
set resolution to lowest setting. depending on your scanner, you ma

# Scratch

In [59]:
display_thread(a_post)

Susan/CA @ 2017-05-25 15:09:00
===PDF files scanned are too large to send in emails. How can..===
I recently invested in a scanner/all in one because I have been getting a lot of full package scan back requests the past several months. I was faxing before. However, when I scan a set of documents (I have also tried breaking up the package into 2 or 3 sections then sending each pdf seperately) then try to email them (on AOL) each time it says the file is too large. How do you all get a large package sent? Is it an AOL issue or should I scan in a different format (JPEG etc.) or better yet how do I create a zip file or some other file so I can sent large scans?Thank you sooooo much!!! 

 Negrete @ 2017-05-25 15:16:00
===Re: PDF files scanned are too large to send in emails. How can..===
AOL is a PIA for trying to send any scanned docs.

 sigtogo/OR @ 2017-05-25 15:22:00
===scanner settings make all the difference for me===
set resolution to lowest setting. depending on your scanner, you ma

In [61]:
req = urllib.request.Request(child_url)
response = urllib.request.urlopen(req)
thePage = response.read()
soup =  BeautifulSoup(thePage, 'lxml')

In [55]:
child_url = 'https://www.notaryrotary.com/forums/forums.asp?id=&forumid=1AAA00000003&messageid=2081515&code=&smsg=&requestid=&action=view&format=threaded'

a_child = Post(child_url)
a_child.fill_post()
a_child.display()

In [130]:
a_post.children

[('1AAA00000003', '2081516')]

In [114]:
class Post:
    
    regex = re.compile("/images/sp.\.gif")
    
    def __init__(self, url):
        self.url = url
        self.parsed_url = urlparse(url)
        self.forum_id = None
        self.message_id = None
        self.author = None
        self.content = None
        self.title = None
        self.time = None
        self.children = []
        
    #process the URL and turn it into soup
    def make_soup(self):
        my_url = self.url
        req = urllib.request.Request(my_url)
        response = urllib.request.urlopen(req)
        thePage = response.read()
        return BeautifulSoup(thePage, 'lxml')

    '''grab post data'''
    def fill_post(self):
        #pull in data
        my_soup = self.make_soup()
        
        
        #get forum_id
        query = parse_qs(self.parsed_url.query)
        self.forum_id = query['forumid'][0]
        self.message_id = int(query['messageid'][0])

        #get title
        self.title = my_soup.find('title').get_text()

        #get content
        self.content = my_soup.find(class_ = 'ForumText').get_text()

        #get author.  There are multiple <b>, but the first is the author
        self.author = my_soup.find('b').get_text()

        #get timestamp
        my_time = my_soup.find('em').get_text()
        my_time = my_time.strip(' ')
        my_time_real = datetime.strptime(my_time, '%m/%d/%y %I:%M%p')
        self.time = datetime.strftime(my_time_real, '%Y-%m-%d %H:%M:00')
        
        #bring in children
        self.list_children(my_soup)
        
    '''direct children only, no grandchildre'''
    def list_children(self, my_soup):
        
        my_level = soup.find(src="/images/triangle.gif")
        my_row = my_level.parent.parent
        #what number?
        my_level_number = self.pull_level_number(my_row)

        my_next = my_row.next_sibling

        self.pick_children(my_next, my_level_number)
        
    def add_child(self, row):
        ra = row.find('a', class_='A2')     #profiles have links, but not A2 class
        rah = ra['href']
        parsed_rah = urlparse(rah)
        rah_query = parse_qs(parsed_rah.query)
        self.children.append(tuple((rah_query['forumid'][0], rah_query['messageid'][0])))
        

    def pull_level_number(self, my_row):
        for i in my_row.find_all('img'):
            if re.match(regex, i['src']):
                return int(i['src'].split('sp')[1][0])
        return 0
                
    def pick_children(self, my_next, my_level_number):
        my_next_level_number = self.pull_level_number(my_next)
        if my_next_level_number == my_level_number + 1:
            self.add_child(my_next)
            try:
                self.pick_children(my_next.next_sibling, my_level_number)  #do on the next row
            except:
                1==1
        elif my_next_level_number > my_level_number + 1:
        #grandchild, don't add but keep moving
            try:
                self.pick_children(my_next.next_sibling, my_level_number)  #do on the next row
            except:
                1==1
        elif my_next_level_number <= my_level_number:
            return  #just end
                
    '''print for validation'''
    def display(self):
        print(self.author + ' @ ' + self.time)
        print('==='+self.title+'===')
        print(self.content)
        print('')


2

In [112]:
import re



In [118]:
def pull_level_number(my_row):
    for i in my_row.find_all('img'):
        if re.match(regex, i['src']):
            return int(i['src'].split('sp')[1][0])
    return 0

def pick_children(my_next, my_level_number):
    my_next_level_number = pull_level_number(my_next)
    if my_next_level_number == my_level_number + 1:
        add_child(my_next)
        try:
            pick_children(my_next.next_sibling, my_next_level_number)  #do on the next row
        except:
            1==1
    elif my_next_level_number > my_level_number + 1:
    #grandchild, don't add but keep moving
        try:
            pick_children(my_next.next_sibling, my_next_level_number)  #do on the next row
        except:
            1==1
    elif my_next_level_number <= my_level_number:
        return  #just end

In [142]:
soup.find

<html><head><meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<title>Re: PDF files scanned are too large to send in emails. How can..</title><meta content="no-cache" http-equiv="Pragma"/><meta content="-1" http-equiv="Expires"/><meta content="Notary Rotary, Inc." name="author"/><meta content="© 2002-2017, Notary Rotary, Inc." name="copyright"/><meta content="Business and Economy" name="classification"/><meta content="General" name="rating"/><meta content="15 days" name="Revisit-After"/><meta content="Re: PDF files scanned are too large to send in emails. How can.." name="description"/><meta content="Re: PDF files scanned are too large to send in emails. How can.." name="keywords"/><meta content="English" http-equiv="Content-Language"/><meta content="document" name="resource-type"/><meta content="2UIYLRAXt4l-90dMVvdHqr-xHQYCwROur90xV5afAaE" name="google-site-verification"/><link href="/include/body.css" rel="stylesheet" type="text/css"/><script typ

In [100]:
pull_level_number(my_row)

NameError: name 'src' is not defined

In [110]:
t = soup.find(class_ = 'TDAlt9')
thread_table = t.parent.parent
for r in thread_table.find_all('a', class_='A2'):
    rh = r['href']
    parsed_rh = urlparse(rh)
    rh_query = parse_qs(parsed_rh.query)
    print(rh_query['forumid'], rh_query['messageid'])

<tr><td align="center" nowrap=""><img height="13" src="/images/triangle.gif" width="12"/></td><td nowrap=""><img src="/images/b0.gif"/> <a class="A2" href="/forums/forums.asp?id=&amp;forumid=1AAA00000003&amp;messageid=2081512&amp;code=&amp;smsg=&amp;requestid=&amp;action=view&amp;format=threaded">PDF files scanned are too large to send in emails. How can..</a> - <font color="#333333" size="2"><b>Susan/CA</b></font><font color="#333333"><em> on 5/25/17 3:09pm  </em></font></td></tr>
<tr><td align="center" nowrap=""></td><td nowrap=""><img src="/images/sp1.gif"/><img src="/images/b0.gif"/> <a class="A2" href="/forums/forums.asp?id=&amp;forumid=1AAA00000003&amp;messageid=2081515&amp;code=&amp;smsg=&amp;requestid=&amp;action=view&amp;format=threaded">Re: PDF files scanned are too large to send in emails. How can..</a> - <font color="#333333" size="2"><b><img border="0" height="10" src="/images/lvl1.gif" width="10"/> <a class="TDAlt7" href="/viewprofile.asp?id=&amp;RID=0116776F851A89E1D874E